# Housing, Health, and Happiness – Milestone P4

## Figure reproduction: Table 4

The goal of this milestone is to reproduce Table 4 of the paper _Housing, Health, and Happiness_.

### 0. Understanding what we'll need

We started this project by reading section V of the paper to understand what part of the data (which columns) will be useful for reproducing the figure.

The data related to the dependant variables can be fount in the following columns:
  + Share of rooms with cement floors (`S_shcementfloor`)
  + Cement floor in kitchen (`S_cementfloorkit`)
  + Cement floor in dining room (`S_cementfloordin`)
  + Cement floor in bathroom (`S_cementfloorbat`)
  + Cement floor in bedroom (`S_cementfloorbed`)

Control and treatment groups are identified by `dpisofirme` (control = 0, treatment = 1).

Model 1 has no control variables.

Model 2 has (25 - 1) control variables:
  + demographic:
    + Number of household members (`S_HHpeople`)
    + (Number of rooms (`S_rooms`) -> This one is mentioned in the paper, but after looking at the STATA file, I noticed it was not used for the regression and decided to drop it)
    + Head of household's years of schooling (`S_headeduc`)
    + Spouse's years of schooling (`S_spouseeduc`)
    + Head of household's age (`S_headage`)
    + Spouse's age (`S_spouseage`)
    + Proportion of Males 0-5yrs in household (`S_dem1`)
    + Proportion of Males 6-17yrs in household (`S_dem2`)
    + Proportion of Males 18-49yrs in household (`S_dem3`)
    + Proportion of Males 50+yrs in household (`S_dem4`)
    + Proportion of Females 0-5yrs in household (`S_dem5`)
    + Proportion of Females 6-17yrs in household (`S_dem6`)
    + Proportion of Females 18-49yrs in household (`S_dem7`)
    + Proportion of Females 50+yrs in household (`S_dem8`)
  + health:
    + Household has animals on land (`S_hasanimals`)
    + Animals allowed to enter the house (`S_animalsinside`)
    + Water connection outside (`S_waterland`)
    + Water connection inside the house (`S_waterhouse`)
    + Electricity (`S_electricity`)
    + Number of times respondent washed hands the day before (`S_washhands`)
    + Uses garbage collection service (`S_garbage`)
    
Model 3 adds 4 control variables:
  + Transfers per capita from government programs (`S_cashtransfers`)
  + Household beneficiary of government milk supplement program (`S_milkprogram`)
  + Household beneficiary of government food program (`S_foodprogram`)
  + Household beneficiary of seguro popular (`S_seguropopular`)
  
All models use `idcluster` for clustering.

Moreover, the paper mentioned dropping samples for which geographical data was available. Initially, I used that as a criteria for filtering the dataset. However, doing this gave worse results than not filtering at all. Thanks to a helpful comment on Zulip, I noticed a discreptancy between the number of samples I was supposed to have according to Table 1 (1362) and the number I had (1187). After looking around, I noticed geographical data was missing for the following 203 lines: (1788:1916), (2505:2527), (2576:2592), (2656:2661), (2755:2782). The last 28 lines seem to correspond to the 28 that were dropped in the paper, but the others seem to be due to some corruption of the dataset for some reason. Therefore, I decided to only drop the final 28 lines in order to use the same dataset as the paper, and I get much better results (which will be discussed in section 3) that way. 

### 1. Imports and data loading

The first step is to import the necessary libraries as well as loading the data, only keeping the necessary rows and columns.

In [ ]:
import pandas as pd
import statsmodels.api as sm
import copy

DATA_PATH = "data/"

In [ ]:
vars_t4 = {
    "treatment_var": ['dpisofirme'],
    "clustering_var": ['idcluster'],
    "demographic_control_vars": ['S_HHpeople', 'S_headeduc', 'S_spouseeduc', 'S_headage', 'S_spouseage', 'S_dem1', 'S_dem2', 'S_dem3', 'S_dem4', 'S_dem5', 'S_dem6', 'S_dem7', 'S_dem8'],
    "health_control_vars": ['S_hasanimals', 'S_animalsinside', 'S_waterland', 'S_waterhouse', 'S_electricity', 'S_washhands', 'S_garbage'],
    "model3_control_vars": ['S_cashtransfers', 'S_milkprogram', 'S_foodprogram', 'S_seguropopular'],
    "dependant_vars": ['S_shcementfloor', 'S_cementfloorkit', 'S_cementfloordin','S_cementfloorbat', 'S_cementfloorbed']
}

models_t4 = {
    'model_1': [],
    'model_2': vars_t4["demographic_control_vars"] + vars_t4["health_control_vars"],
}
models_t4['model_3'] = models_t4['model_2'] + vars_t4["model3_control_vars"]

### 2. Generating missing values

Missing values are replaced by 0 and a dummy variable indicating whether the value was missing is added (missing=1, present=0) for each variable containing missing values (others would only contain 0). The models are also updated to take the dummy variables into account.

In [ ]:
#Parameter inplace is used to show explicitly that this function has side effects on df
def generateMissingValues(df, models_, inplace):
    models = copy.deepcopy(models_)
    columns = models["model_3"]
    if(inplace == False):
        raise ValueError("Parameter inplace has to be true")
    for col in columns:
        if df[col].isnull().values.any():
            df['dmiss_' + col] = df[col].apply(pd.isna).apply(float)
            if col in models['model_2']:
                models['model_2'].append('dmiss_' + col)
            models['model_3'].append('dmiss_' + col)
    df.fillna(0, inplace=True)
    return models

### 3. Regression

There are 2 steps to this part.

The first is to compute the mean and standard deviation for the control group for each dependant variable. This is done using `mean()` and `std()` on the control DataFrame (i.e. `dpisofirme` = 0).

The second part is to do a linear regression for each dependent variable once for each model. This is his done using 2 nested for loops (over models, then over dependent variables) and using `statsmodels`'s `OLS` with a cluster covariance estimator (`idcluster`).

In [ ]:
### Helper function to convert p-value to stars like in the paper
def to_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "** "
    elif p < 0.1:
        return "*  "
    return ""

In [ ]:
def computeResults(df, models, vs):
    # Part 1: control
    dependant_vars = vs["dependant_vars"]
    control = df[df.dpisofirme == 0][dependant_vars]
    res = pd.DataFrame({
        'control means': control.mean(),
        'control std': control.std()
    }, index=dependant_vars)
    
    # Part 2: linear regression
    Y = df[dependant_vars]
    
    for k, v in models.items():
        X = df[vs["treatment_var"] + v]
        X = sm.add_constant(X)
        column = []
        for label, y in Y.items():
            regression = sm.OLS(y, X).fit(cov_type='cluster',
                                          cov_kwds={'groups': df[vs["clustering_var"]]})
            coeff = regression.params['dpisofirme']
            significance = to_stars(regression.pvalues['dpisofirme'])
            column.append((coeff, regression.bse['dpisofirme'], significance, 
                           100 * coeff / res.loc[label]['control means']))
        res[k] = column
    
    return pd.DataFrame(res, index=dependant_vars)

### 4. Showing and discussing the results

I display the DataFrame containing the results rounded to 3 decimals, then compute the difference (after rounding) with the results from the paper. 

The differences is fairly small (~10e-3 for the coeffs) for models 2 and 3. This may be due to the way I handled missing values in the control variables.

In [ ]:
import numbers as nb
def round3(val):
    if isinstance(val, nb.Number):
        return round(val, 3)
    elif isinstance(val, str):
        return val
    else:
        tpe = type(val)
        return tpe(map(round3, val))

def round_res(df, vs):
    res = df.apply(round3)
    res.index = vs["dependant_vars"]
    return res

In [ ]:
expected_res = pd.DataFrame({'control means': [0.728, 0.671, 0.709, 0.803, 0.668],
                             'control std': [0.363, 0.470, 0.455, 0.398, 0.471],
                             'model_1': [(0.202, 0.021, '***', 27.746),
                                         (0.255, 0.025, '***', 37.936),
                                         (0.210, 0.026, '***', 29.633),
                                         (0.105, 0.022, '***', 13.071),
                                         (0.238, 0.020, '***', 35.598)],
                             'model_2': [(0.208, 0.019, '***', 28.512),
                                         (0.260, 0.023, '***', 38.708),
                                         (0.217, 0.025, '***', 30.588),
                                         (0.113, 0.018, '***', 14.043),
                                         (0.245, 0.021, '***', 36.735)],
                             'model_3':[(0.210, 0.019, '***', 28.876),
                                        (0.265, 0.023, '***', 39.440),
                                        (0.221, 0.025, '***', 31.189),
                                        (0.117, 0.018, '***', 14.536),
                                        (0.245, 0.020, '***', 36.695)]},
                            index = vars_t4["dependant_vars"])

In [ ]:
import operator
#x, y chars
def star_compare(x, y):
    values = {' ', '*'}
    if(x not in values or y not in values):
        return '/'
    bx = x == '*'
    by = y == '*'
    res = [[' ', '-'], ['+', '*']]
    return res[bx][by]

def f(x, y):
    if isinstance(x, tuple):
        return tuple(map(f, x, y))
    elif not (isinstance(x, str)):
        return "{:.2e}".format(operator.sub(x, y))
    else:
        return "".join(map(star_compare, list(x),list(y)))
    
def diff(df1, df2):
    comp = df1.copy()
    for col in comp.columns:
        comp[col] = list(map(f, comp[col], df2[col]))
    return comp

In [ ]:
df_t4 = pd.read_stata(DATA_PATH + "PisoFirme_AEJPol-20070024_household.dta")[:2755]
df_t4 = df_t4[vars_t4["treatment_var"] + vars_t4["clustering_var"] + models_t4['model_3'] + vars_t4["dependant_vars"]]
new_models_t4 = generateMissingValues(df_t4, models_t4, inplace=True)

res = computeResults(df_t4, new_models_t4, vars_t4)

rounded_res = round_res(res, vars_t4)
display(rounded_res)

display(expected_res)
comp = diff(expected_res, rounded_res)
display(comp)

## Figure reproduction: Table 5

The goal of this milestone is to reproduce Table 5 of the paper _Housing, Health, and Happiness_.

### 0. Understanding what we'll need

We started this replication exercise by adapting our replication of Table 4. The main differences are:
1. we use the `individual.dta` rather than the `household.dta` file;
1. we use different dependant variables.

Note that Table 5 only focuses on children under the age of 6, so we drop every row pertaining to a person older than 6 years old (see [Annex A](#annexA) for further justification).

Using the explanation in section V of the paper as well as the STATA code, we identified the parts of the data which would be useful for reproducing the figure.

The data related to the dependant variables can be found in the following columns:
  + Parasite count (`S_parcount`)
  + Diarrhea (`S_diarrhea`)
  + Anemia (`S_anemia`)
  + McArthur Communication Development Test score (`S_mccdts`)
  + Picture Peabody Vocabulary Test percentile score (`S_pbdypct`)
  + Height-for-age z-score (`S_haz`)
  + Weight-for-height z-score (`S_whz`)

Control and treatment groups are identified by `dpisofirme` (control = 0, treatment = 1).

Model 1 has no control variables.

Model 2 has 58 control variables:
  + demographic:
    + Number of household members (`S_HHpeople`)
    + Number of rooms (`S_rooms`)
    + Age (`S_age`)
    + Male (`S_gender`) -> the README specifies that Male = 1, but the loaded dataframe contained the values `0.0` and `hombre`, so this was corrected to be `0` and `1`
    + Mother of at least one child in household present (`S_childma`)
    + Mother's age (if present) (`S_childmaage`)
    + Mother's years of schooling (if present) (`S_childmaeduc`)
    + Father of at least one child in household present (`S_childpa`)
    + Father's age (if present) (`S_childpaage`)
    + Father's years of schooling (if present) (`S_childpaeduc`)
    + (Trimester * Gender) Dummy for children 0-5yrs (`dtriage*`) [48]
  + health:
    + Household has animals on land (`S_hasanimals`)
    + Animals allowed to enter the house (`S_animalsinside`)
    + Water connection outside (`S_waterland`)
    + Water connection inside the house (`S_waterhouse`)
    + Electricity (`S_electricity`)
    + Number of times respondent washed hands the day before (`S_washhands`)
    + Uses garbage collection service (`S_garbage`)
    
Model 3 adds 4 control variables:
  + Transfers per capita from government programs (`S_cashtransfers`)
  + Household beneficiary of government milk supplement program (`S_milkprogram`)
  + Household beneficiary of government food program (`S_foodprogram`)
  + Household beneficiary of seguro popular (`S_seguropopular`)
  
All models use `idcluster` for clustering.

Moreover, the paper mentioned dropping samples for which geographical data was unavailable. In the first replication, we noticed that there were multiple clusters of rows for which geographical data was missing, but that we got the correct number of datapoints and much better results by only dropping the final cluster, so we tried applying the same method here. Since the paper never mentions how many individuals' data they used, we used the data provided in Table 1 for each dependant variable as a point of comparison. The list of rows with missing data can be found in [Annex B](#annexB), and the check of the number of datapoints after dropping the final cluster is done in [Annex C](#annexC).

### 1. Imports and data loading

The first step is to import the necessary libraries as well as loading the data, only keeping the necessary rows and columns.

In [ ]:
import pandas as pd
import statsmodels.api as sm

DATA_PATH = "data/"

In [ ]:
original_df = pd.read_stata(DATA_PATH + "PisoFirme_AEJPol-20070024_individual.dta")
children_df = original_df[original_df.S_age < 6].reset_index(drop=True)
children_df['S_gender'] = children_df['S_gender'].apply(lambda x: x == 'hombre').astype(int)
df = children_df[:4052] # Magic number! See Annex B!

treatment_var =  ['dpisofirme']
clustering_var = ['idcluster']
demographic_control_vars_1 = ['S_HHpeople', 'S_rooms', 'S_age', 'S_gender', 'S_childma', 'S_childmaage',
                            'S_childmaeduc', 'S_childpa', 'S_childpaage', 'S_childpaeduc']
demographic_control_vars_2 = [x for x in df.columns if 'dtriage' in x]
health_control_vars = ['S_hasanimals', 'S_animalsinside', 'S_waterland', 'S_waterhouse', 
                  'S_electricity', 'S_washhands', 'S_garbage']
model3_control_vars = ['S_cashtransfers', 'S_milkprogram', 'S_foodprogram', 'S_seguropopular']
dependant_vars = ['S_parcount', 'S_diarrhea', 'S_anemia', 'S_mccdts', 'S_pbdypct', 'S_haz', 'S_whz']
models = {
    'model_1': [],
    'model_2': demographic_control_vars_1 + demographic_control_vars_2 + health_control_vars,
    'model_3': demographic_control_vars_1 + demographic_control_vars_2 + health_control_vars + model3_control_vars
}

df = df[treatment_var + clustering_var + dependant_vars + models['model_3'] + ['coord_x']]
df

### 2. Generating missing values

Missing values are replaced by 0 and a dummy variable indicating whether the value was missing is added (missing=1, present=0) for each independant variable containing missing values (others would only contain 0). The models are also updated to take the dummy variables into account.

Note: in the STATA file the paper's authors do not check for missing values in `dtriage` colums. Here, we confirm that there is no missing values in these columns, so we do not need to manually exclude them from the `values` dict.

In [ ]:
columns = models["model_3"]
values = dict(zip(columns, [0] * len(columns)))

print('Columns with missing values:')
for col in columns:
    if df[col].isnull().values.any():
        print(col)
        df['dmiss_' + col] = df[col].apply(pd.isna).apply(int)
        if col in models['model_2']:
            models['model_2'].append('dmiss_' + col)
        models['model_3'].append('dmiss_' + col)
df.fillna(values, inplace=True)

### 3. Regression

There are 2 steps to this part.

The first is to compute the mean and standard deviation for the control group for each dependant variable. This is done using `mean()` and `std()` on the control DataFrame (i.e. `dpisofirme` = 0).

The second part is to do a linear regression for each dependent variable once for each model. This is his done using 2 nested for loops (over models, then over dependent variables) and using `statsmodels`'s `OLS` with a cluster covariance estimator (`idcluster`). Since we have missing values for certain rows in the dependant variables, we drop them for the regression.

In [ ]:
# Part 1: control
control = df[df.dpisofirme == 0][dependant_vars]
results = pd.DataFrame({
    'control means': control.mean(),
    'control std': control.std()
}, index=dependant_vars)
results

In [ ]:
### Helper function to convert p-value to stars like in the paper
def to_stars(p):
    if p < 0.01:
        return "***"
    elif p < 0.05:
        return "**"
    elif p < 0.1:
        return "*"
    return ""

In [ ]:
# Part 2: linear regression
Y = df[dependant_vars]

for k, v in models.items():
    X = df[treatment_var + v]
    X = sm.add_constant(X)
    column = []
    for label, y in Y.items():
        regression = sm.OLS(y, X, missing='drop').fit(cov_type='cluster',
                                      cov_kwds={'groups': df.dropna(subset=[label])[clustering_var]})
        coeff = regression.params['dpisofirme']
        significance = to_stars(regression.pvalues['dpisofirme'])
        column.append((coeff, regression.bse['dpisofirme'], significance,
                       100 * coeff / results.loc[label]['control means']))
    results[k] = column

### 4. Showing and discussing the results

We display the DataFrame containing the results rounded to 3 decimals, then compute the difference (after rounding) with the results from the paper. 

The differences is fairly small (~10e-3 for the coeffs) for models 2 and 3. 

In [ ]:
import numbers as nb
def round3(val):
    if isinstance(val, nb.Number):
        return round(val, 3)
    elif isinstance(val, str):
        return val
    else:
        tpe = type(val)
        return tpe(map(round3, val))

def round_res(df):
    res = df.apply(round3)
    res.index = dependant_vars
    # print(res)
    return res # printing is somehow prettier this way 
round_res(results)

In [ ]:
expected_res = pd.DataFrame({'control means': [0.333, 0.142, 0.426, 13.354, 30.656, -0.605, 0.125],
                             'control std': [0.673, 0.349, 0.495, 18.952, 24.864, 1.104, 1.133],
                             'model_1': [(-0.065, 0.032, '**', -19.545),
                                         (-0.018, 0.009, '*', -12.819),
                                         (-0.085, 0.028, '***', -20.059), 
                                         (4.031, 1.650, '**', 30.182), 
                                         (2.668, 1.689,'' , 8.702), 
                                         (0.007, 0.043, '', -1.161), 
                                         (0.002, 0.034, '', 1.790)],
                             'model_2': [(-0.064, 0.031, '**', -19.345),
                                         (-0.020, 0.009, '**', -13.834),
                                         (-0.081, 0.027, '***', -18.908),
                                         (5.652, 1.642, '***', 42.325),
                                         (3.206, 1.430, '**', 10.460),
                                         (0.002, 0.038, '',0.279),
                                         (-0.005, 0.036, '', -4.119)],
                             'model_3':[(-0.064, 0.032, '**', -19.198),
                                         (-0.018, 0.009, '*', -12.803),
                                         (-0.083, 0.027, '***', -19.388),
                                         (5.557, 1.641, '***', 41.609),
                                         (3.083, 1.410, '**', 10.058),
                                         (-0.002, 0.039, '', -0.323),
                                         (-0.011, 0.037, '', -8.727)]},
                            index = dependant_vars)
expected_res

In [ ]:
import operator
def compare_results(x, y):
    if isinstance(x, tuple):
        return tuple(map(compare_results, x, y))
    elif not (isinstance(x, str)):
        return "{:.2e}".format(operator.sub(x, y))
    else:
        return x == y
comp = expected_res.copy()
res = round_res(results)
for col in comp.columns:
    comp[col] = list(map(compare_results, comp[col], res[col]))
comp

<span id="annexA"></span>
### Annex A

In [ ]:
list_vars = []
olderthan6_df = original_df.drop(children_df.index)[dependant_vars]
print('Checking if all dependant variables are `NaN` for individuals older than 6...')
if (olderthan6_df.apply(pd.isna).values.all()):
    s = ""
else:
    s = "not"
print(f'We can{s} drop all aforementioned rows')

<span id="annexB"></span>
### Annex B

In [ ]:
nans = children_df.loc[pd.isna(children_df['coord_x'])].index
ranges = []
low, up = nans[0], nans[1]
for i in range(len(nans) - 1):
    up = nans[i]
    if (up + 1 != nans[i+1] or i+1 == len(nans) - 1):
        if (i+1 == len(nans) - 1):
            up = nans[i+1]
        ranges.append((low, up))
        low = nans[i+1]
ranges_str = ""
for (l, r) in ranges:
    ranges_str += '[' + str(l) + ', ' +  str(r) + ']\n'
print(f'Intervals of rows with missing geographical data:\n{ranges_str}')

<span id="annexC"></span>
### Annex C

In [ ]:
expected_amount = pd.DataFrame({'treat_expected': [1528, 1930, 1768, 291, 757, 1865, 1881],
                                'cont_expected' : [1566, 2105, 1951, 302, 817, 2053, 2058]},
                              index=dependant_vars)
treat, cont, drop_t, drop_c = [], [], [], []
treat_df = df[df.dpisofirme == 1]
cont_df = df[df.dpisofirme == 0]

for col in dependant_vars:
    dt = len(treat_df.loc[pd.isna(treat_df[col])])
    treat.append(len(treat_df) - dt)
    drop_t.append(dt)
    dc = len(cont_df.loc[pd.isna(cont_df[col])])
    cont.append(len(cont_df) - dc)
    drop_c.append(dc)
expected_amount['treat'] = treat
expected_amount['cont'] = cont
expected_amount['dropped_treat'] = drop_t
expected_amount['dropped_cont'] = drop_c
expected_amount['dropped_tot'] = expected_amount['dropped_treat'] + expected_amount['dropped_cont']
expected_amount['delta_t'] = expected_amount.treat_expected - expected_amount.treat
expected_amount['delta_c'] = expected_amount.cont_expected - expected_amount.cont
expected_amount['tot'] = expected_amount.treat + expected_amount.cont
expected_amount